# LightGBM pCTR Estimation
Train a LightGBM model (https://lightgbm.readthedocs.io/en/latest/ , https://github.com/Microsoft/LightGBM) to estimate pCTRs

In [1]:
#Import libraries
import lightgbm as lgb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import math
import collections as coll

from glob import glob
from scipy import stats
from IPython.display import display
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score,confusion_matrix,jaccard_similarity_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error


np.random.seed(20)

### Data loading & feature engineering

In [2]:
# Define recalibration function
def recalibratePred(predictions, factor):
    return predictions/(predictions+(1-predictions)/factor)

In [3]:
%%time
#Defining directory.
#Path = '/Users/olivierobalbinetti/Desktop/University College London/Term 2'\
#       '/Multi Agents Artificial Intelligence/Courseworks/Group coursework/'\
#       'Data/Original'

Path = 'C://Datasets//Multiagent//rtb'

#Importing data.
Datasets = {}
os.chdir(Path)
for Filename in glob('*.csv'):
    Datasets[Filename[:-4].title()] = pd.read_csv(Filename, sep = ',')
    
#Defining variables.
Test = Datasets['Test']
Train = Datasets['Train']
Validation = Datasets['Validation']
Total_data = sum([len(Datasets[key]) for key in Datasets.keys()])

#Feature engineering: ad exchange.
Test['adexchange'] = Test['adexchange'].fillna('nan')
Train['adexchange'] = Train['adexchange'].fillna('nan')
Validation['adexchange'] = Validation['adexchange'].fillna('nan')

#Feature engineering: slot size.
Test['slotsize'] = Test['slotwidth'].astype('str') + '*' +\
                   Test['slotheight'].astype('str')
Train['slotsize'] = Train['slotwidth'].astype('str') + '*' +\
                    Train['slotheight'].astype('str')
Validation['slotsize'] = Validation['slotwidth'].astype('str') + '*' +\
                         Validation['slotheight'].astype('str')

#Feature engineering: browser user agent.
Temp = Train['useragent'].str.split('_', n=1, expand=True)
Train['OS'] = Temp[0]
Train['Browser'] = Temp[1]

Temp = Test['useragent'].str.split('_', n=1, expand=True)
Test['OS'] = Temp[0]
Test['Browser'] = Temp[1]

Temp = Validation['useragent'].str.split('_', n=1, expand=True)
Validation['OS'] = Temp[0]
Validation['Browser'] = Temp[1]

#Feature engineering: user tags.
Train['usertag'] = Train['usertag'].fillna('nan')                      #Train.
Train['usertag list'] = Train['usertag'].str.split(',').values
Temp = list(Train['usertag list'].values)
Tags = [item for sublist in Temp for item in sublist]

Dict = dict(coll.Counter(Tags).most_common())
Dict['nan'] = max(Dict.values())+1
Dict = dict(sorted(Dict.items(), key=lambda kv: kv[1], reverse=True))
Dict = dict(enumerate(Dict.keys()))

Dict = {value:key for key,value in Dict.items()}
Dict['nan'] = 'nan'
Train = Train.assign(mapped=[[Dict[k] for k in row if Dict.get(k)]
                             for row in Train['usertag list'].values])


Test['usertag'] = Test['usertag'].fillna('nan')                         #Test.
Test['usertag list'] = Test['usertag'].str.split(',').values
Temp = list(Test['usertag list'].values)
Tags = [item for sublist in Temp for item in sublist]

Dict = dict(coll.Counter(Tags).most_common())
Dict['nan'] = max(Dict.values())+1
Dict = dict(sorted(Dict.items(), key=lambda kv: kv[1], reverse=True))
Dict = dict(enumerate(Dict.keys()))

Dict = {value:key for key,value in Dict.items()}
Dict['nan'] = 'nan'
Test = Test.assign(mapped=[[Dict[k] for k in row if Dict.get(k)]
                           for row in Test['usertag list'].values])


Validation['usertag'] = Validation['usertag'].fillna('nan')       #Validation.
Validation['usertag list'] = Validation['usertag'].str.split(',').values
Temp = list(Validation['usertag list'].values)
Tags = [item for sublist in Temp for item in sublist]

Dict = dict(coll.Counter(Tags).most_common())
Dict['nan'] = max(Dict.values())+1
Dict = dict(sorted(Dict.items(), key=lambda kv: kv[1], reverse=True))
Dict = dict(enumerate(Dict.keys()))

Dict = {value:key for key,value in Dict.items()}
Dict['nan'] = 'nan'
Validation = Validation.assign(mapped=[[Dict[k] for k in row if Dict.get(k)]
                                       for row in Validation['usertag list'].values])

#Feature engineering: renaming.
Test = Test.rename(columns={'mapped':'tagcodes'})
Train = Train.rename(columns={'mapped':'tagcodes'})
Validation = Validation.rename(columns={'mapped':'tagcodes'})

Wall time: 1min 46s


#### Downsample train set, given class imbalance

In [4]:
%%time


#Random downsample for training before more feature engineering 
downsample_factor = 0.00075
not_clicked_idx = np.where(Train["click"] ==0)[0]
clicked_idx = np.where(Train["click"] ==1)[0]
print("Original ratio",len(clicked_idx),len(not_clicked_idx))
print("Downsample factor:",downsample_factor)
print("New factor", len(clicked_idx)/(len(not_clicked_idx)*downsample_factor))
not_clicked_down_idx = np.random.choice(not_clicked_idx,size=(int(len(not_clicked_idx)*downsample_factor)),replace=True)
idx_concat = np.concatenate((clicked_idx,not_clicked_down_idx))

Train_subset = Train.loc[idx_concat]

#User tag columns for Train & Validation
usertag_dummies = Train_subset['usertag list'].str.join('|').str.get_dummies().add_prefix('tags_')
usertag_dummies.drop(columns=['tags_nan'], inplace=True)
Train_subset = pd.concat([Train_subset,usertag_dummies], axis=1)

usertag_dummies = Validation['usertag list'].str.join('|').str.get_dummies().add_prefix('tags_')
usertag_dummies.drop(columns=['tags_nan'], inplace=True)
Validation = pd.concat([Validation,usertag_dummies], axis=1)

usertag_dummies = Test['usertag list'].str.join('|').str.get_dummies().add_prefix('tags_')
usertag_dummies.drop(columns=['tags_nan'], inplace=True)
Test = pd.concat([Test,usertag_dummies], axis=1)

Original ratio 1793 2429188
Downsample factor: 0.00075
New factor 0.9841423004998652
Wall time: 52.2 s


#### Create encoded  variables on train subset
Given that LGBM does not accept string variables, encode them into numerical categorical variables.

In [5]:
%%time
# Change categorical string variables to int
# Error in columns: adexchange, domain, slotid, slotvisibility, slotformat, creative, keypage, OS, Browser

#creative, keypage, slotformat, slotvisibility
def encodeColumn(columnName):
    Train_subset[columnName+'_enc']  = Train_subset[columnName]    
    Validation[columnName+'_enc'] =Validation[columnName]    
    Test[columnName+'_enc']=Test[columnName]    
    
    labels = Train[columnName].astype('category').cat.categories.tolist()    
    label_enc = {columnName+'_enc' : {k: v for k,v in zip(labels,list(range(1,len(labels)+1)))}}
    Train_subset.replace(label_enc, inplace=True)
    Validation.replace(label_enc, inplace=True)
    Test.replace(label_enc, inplace=True)
    
    Validation[columnName+'_enc'] =Validation[columnName+'_enc'].astype('category')
    Test[columnName+'_enc']= Test[columnName+'_enc'].astype('category')
    Train_subset[columnName+'_enc'] = Train_subset[columnName+'_enc'].astype('category')
    
columns_to_encode =  {'creative','keypage','slotformat','slotvisibility','advertiser','OS','Browser'}
for column in columns_to_encode:
    encodeColumn(column)   
    


Wall time: 12.4 s


#### Define columns to use
Out of the columns on train subset and the validation set, find the common ones. Then, from those, drop out those that are unique or with little value

In [7]:
t1_columns = (Train_subset.columns.values)
t2_columns = Validation.columns.values
#t1_columns[t1_columns.c]
common = np.intersect1d(t1_columns,t2_columns)
#print(common)
dropped_features = {'Browser', 'creative', 'keypage', 'OS','slotformat' ,'slotvisibility', 'advertiser',
                    'domain', 'usertag', 'usertag list','tagcodes',  'useragent',  #Encoded features
                    'IP', 'bidid', 'click', 'bidprice','url', 'urlid', 'slotid','userid','payprice', 'domain', #Unique values                    
                    'slotsize','adexchange','slotheight', 'slotwidth'  #Potential
                   } 
                    #,'slotformat',,'slotformat_enc', 'slotprice',, 
new_columns =  [x for x in common if x not in dropped_features]
print(new_columns)

['Browser_enc', 'OS_enc', 'advertiser_enc', 'city', 'creative_enc', 'hour', 'keypage_enc', 'region', 'slotformat_enc', 'slotprice', 'slotvisibility_enc', 'tags_10006', 'tags_10024', 'tags_10031', 'tags_10048', 'tags_10052', 'tags_10057', 'tags_10059', 'tags_10063', 'tags_10067', 'tags_10074', 'tags_10075', 'tags_10076', 'tags_10077', 'tags_10079', 'tags_10083', 'tags_10093', 'tags_10102', 'tags_10110', 'tags_10111', 'tags_10114', 'tags_10115', 'tags_10116', 'tags_10117', 'tags_10120', 'tags_10123', 'tags_10125', 'tags_10126', 'tags_10127', 'tags_10129', 'tags_10131', 'tags_10133', 'tags_10138', 'tags_10140', 'tags_10142', 'tags_10145', 'tags_10146', 'tags_10147', 'tags_10148', 'tags_10149', 'tags_10684', 'tags_11092', 'tags_11278', 'tags_11379', 'tags_11423', 'tags_11512', 'tags_11576', 'tags_11632', 'tags_11680', 'tags_11724', 'tags_11944', 'tags_13042', 'tags_13403', 'tags_13496', 'tags_13678', 'tags_13776', 'tags_13800', 'tags_13866', 'tags_13874', 'tags_14273', 'tags_16593', 'tags_

#### Select train and validation data
Based on the selected columns above, select data for training and validation

In [8]:
%%time

# Different sets of manually selected features

selected_features = [new_columns]    
feat_sel = 0

#Get train and validation subsets

x_train = Train_subset[selected_features[feat_sel]]
y_train = Train_subset["click"]

x_val = Validation[selected_features[feat_sel]]
y_val = Validation["click"]

x_test = Test[selected_features[feat_sel]]


Wall time: 229 ms


### Train LightGBM model

In [9]:
%%time
d_train = lgb.Dataset(x_train, label=y_train)
d_validation = lgb.Dataset(x_val, label=y_val)

#learning_rate': 0.1, 'max_depth': 5, 'min_data': 100, 'min_data_in_leaf': 30, 'n_estimators': 100, 'num_leaves': 50

params = {} # LightGBM parameters
params['learning_rate'] = 0.3
params['boosting_type'] = 'dart'
params['objective'] = 'binary'
params['metric'] = 'auc'
params['sub_feature'] = 0.5
params['num_leaves'] = 60
params['min_data'] = 500
params['max_depth'] = 5
params['min_data_in_leaf'] = 20
params['early_stopping_rounds'] = 30
#params['categorical_feature'] = 'auto'
#params['drop_rate'] = 0.95
clf = lgb.train(params, d_train, 200, valid_sets= [d_validation],verbose_eval=7)

#Prediction
y_pred_val=clf.predict(x_val)

y_pred_val_bin = np.zeros(len(y_pred_val))
for i in range(len(y_pred_val)):
    if y_pred_val[i]>=.5:       # setting threshold to .5
        y_pred_val_bin[i]=1
    else:  
        y_pred_val_bin[i]=0

print("ROC AUC score", roc_auc_score(y_val,y_pred_val))
print("Confusion Matrix\n", confusion_matrix(y_val, y_pred_val_bin))


C:\Users\mau_c\Anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\mau_c\Anaconda3\lib\site-packages\lightgbm\basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 30 rounds.
[7]	valid_0's auc: 0.846954
[14]	valid_0's auc: 0.864436
[21]	valid_0's auc: 0.876692
[28]	valid_0's auc: 0.878339
[35]	valid_0's auc: 0.881739
[42]	valid_0's auc: 0.882191
[49]	valid_0's auc: 0.883084
[56]	valid_0's auc: 0.882803
[63]	valid_0's auc: 0.87683
[70]	valid_0's auc: 0.879159
[77]	valid_0's auc: 0.876438
Early stopping, best iteration is:
[50]	valid_0's auc: 0.883785
ROC AUC score 0.8851077109963048
Confusion Matrix
 [[252282  51441]
 [    49    153]]
Wall time: 11.9 s


### Predict Test pCTRs
Based on the trained model, predict Test pCTRs and write to file

In [12]:
# Get pCTRs for Validation and Test Dataset
Path = 'C://Datasets//Multiagent//calc//pctrs'
os.chdir(Path)

y_pred_test=clf.predict(x_test)
#print(len(y_real[y_real<0.5]))

# Recalibrate predictions
y_pred_test_recal = recalibratePred(y_pred_test,downsample_factor)
y_pred_val_recal =  recalibratePred(y_pred_val, downsample_factor)

# Write Test pCTR
Filename = 'LGBM_test_pCTR' + '.csv'
Header = 'bidid,pCTR'
with open(Filename, 'w') as file:
    file.write(Header + '\n')
    for request in range(0, len(Test)):
        file.write(Test['bidid'].iloc[request] + ',' + 
                   str(y_pred_test_recal[request]) + '\n')

# Write validation pCTR 
Filename = 'LGBM_validation_pCTR' + '.csv'
Header = 'bidid,pCTR'
with open(Filename, 'w') as file:
    file.write(Header + '\n')
    for request in range(0, len(Validation)):
        file.write(Validation['bidid'].iloc[request] + ',' + 
                   str(y_pred_val_recal[request]) + '\n')
print("Files created")

Files created


#### Playzone
Hyperparameter optimisation, feature importance

In [13]:
#LightGBM Regressor

def rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False

def roc_auc_f(y_true, y_pred):
    return 'ROC_AUC', roc_auc_score(y_true, y_pred),False


gbm_1 = lgb.LGBMClassifier(
    task= 'train',
    boosting_type= 'dart',
    objective= 'binary',
    metric= {'binary_logloss','auc'},
    sub_feature = 0.5,
    num_leaves= 60,
    min_data = 500,
    max_depth=5,
    min_data_in_leaf = 30,
    learning_rate= 0.3,
    n_estimators = 200
)
                         #feature_fraction= 0.9,
                         #bagging_fraction= .9,
                         #bagging_freq= 70,
                         #verbose= 100

gbm_1.fit(x_train, y_train,eval_set=[(x_val, y_val)],eval_metric=roc_auc_f)#,early_stopping_rounds=35

y_pred_gbm1 = gbm_1.predict(x_val, num_iteration=gbm_1.best_iteration_)
print('The rmse of prediction is:', mean_squared_error(y_val, y_pred_gbm1) ** 0.5)

# feature importances
print('Feature importances:', list(gbm_1.feature_importances_))


[1]	valid_0's auc: 0.707528	valid_0's binary_logloss: 0.653183	valid_0's ROC_AUC: 0.707528
[2]	valid_0's auc: 0.785125	valid_0's binary_logloss: 0.604679	valid_0's ROC_AUC: 0.785125
[3]	valid_0's auc: 0.797203	valid_0's binary_logloss: 0.573987	valid_0's ROC_AUC: 0.797203
[4]	valid_0's auc: 0.795486	valid_0's binary_logloss: 0.556988	valid_0's ROC_AUC: 0.795486
[5]	valid_0's auc: 0.806473	valid_0's binary_logloss: 0.547438	valid_0's ROC_AUC: 0.806473
[6]	valid_0's auc: 0.836789	valid_0's binary_logloss: 0.51234	valid_0's ROC_AUC: 0.836789
[7]	valid_0's auc: 0.838553	valid_0's binary_logloss: 0.509292	valid_0's ROC_AUC: 0.838553
[8]	valid_0's auc: 0.83513	valid_0's binary_logloss: 0.52589	valid_0's ROC_AUC: 0.83513
[9]	valid_0's auc: 0.856773	valid_0's binary_logloss: 0.495693	valid_0's ROC_AUC: 0.856773
[10]	valid_0's auc: 0.854532	valid_0's binary_logloss: 0.495579	valid_0's ROC_AUC: 0.854532
[11]	valid_0's auc: 0.860075	valid_0's binary_logloss: 0.482525	valid_0's ROC_AUC: 0.860075
[

[91]	valid_0's auc: 0.872392	valid_0's binary_logloss: 0.434626	valid_0's ROC_AUC: 0.872392
[92]	valid_0's auc: 0.872612	valid_0's binary_logloss: 0.43298	valid_0's ROC_AUC: 0.872612
[93]	valid_0's auc: 0.872923	valid_0's binary_logloss: 0.43178	valid_0's ROC_AUC: 0.872923
[94]	valid_0's auc: 0.873245	valid_0's binary_logloss: 0.43154	valid_0's ROC_AUC: 0.873245
[95]	valid_0's auc: 0.873386	valid_0's binary_logloss: 0.43203	valid_0's ROC_AUC: 0.873386
[96]	valid_0's auc: 0.873686	valid_0's binary_logloss: 0.432279	valid_0's ROC_AUC: 0.873686
[97]	valid_0's auc: 0.87396	valid_0's binary_logloss: 0.428817	valid_0's ROC_AUC: 0.87396
[98]	valid_0's auc: 0.873435	valid_0's binary_logloss: 0.429658	valid_0's ROC_AUC: 0.873435
[99]	valid_0's auc: 0.874371	valid_0's binary_logloss: 0.429552	valid_0's ROC_AUC: 0.874371
[100]	valid_0's auc: 0.87375	valid_0's binary_logloss: 0.428653	valid_0's ROC_AUC: 0.87375
[101]	valid_0's auc: 0.873689	valid_0's binary_logloss: 0.429696	valid_0's ROC_AUC: 0.8

In [36]:
def roc_auc_f(y_true, y_pred):
    return 'ROC_AUC', roc_auc_score(y_true, y_pred),False


estimator = lgb.LGBMClassifier(                             
                               task= 'train',
                               boosting_type= 'gbdt',
                               objective= 'binary',
                               n_estimators = 100,
                               )

param_grid_esy = {
    'learning_rate': [0.3],
    'n_estimators': [100],
    'num_leaves':[60],
    'min_data_in_leaf':[20],
    'max_depth':[5],
    'random_state' : [20], 
    'min_data':[500],
    'sub_feature':[0.5],
    'metric':['auc']
}
param_grid = {
    'learning_rate': [0.01, 0.1, .2,.3,.4,.02,.05],
    'n_estimators': [100],
    'num_leaves':[50,80,100,120],
    'min_data_in_leaf':[15,25,30],
    'max_depth':[5,8],
    'min_data':[100,500,1000]    
}

gbm2 = GridSearchCV(estimator, param_grid, cv=3, verbose=4, refit='AUC')
gbm2.fit(x_train, y_train,eval_metric='roc_auc')
#print('Best parameters found by grid search are:', gbm2.best_params_)
print(gbm2.best_params_, gbm2.best_score_, gbm2.best_estimator_)

print("ROC AUC score", roc_auc_score(y_val,gbm2.predict(x_val)))


Fitting 3 folds for each of 504 candidates, totalling 1512 fits
[CV] learning_rate=0.01, max_depth=5, min_data=100, min_data_in_leaf=15, n_estimators=100, num_leaves=50 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  learning_rate=0.01, max_depth=5, min_data=100, min_data_in_leaf=15, n_estimators=100, num_leaves=50, score=0.7394190871369295, total=   0.1s
[CV] learning_rate=0.01, max_depth=5, min_data=100, min_data_in_leaf=15, n_estimators=100, num_leaves=50 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  learning_rate=0.01, max_depth=5, min_data=100, min_data_in_leaf=15, n_estimators=100, num_leaves=50, score=0.7775933609958506, total=   0.1s
[CV] learning_rate=0.01, max_depth=5, min_data=100, min_data_in_leaf=15, n_estimators=100, num_leaves=50 


...

[CV]  learning_rate=0.05, max_depth=8, min_data=100, min_data_in_leaf=25, n_estimators=100, num_leaves=100, score=0.7560165975103734, total=   0.2s
[CV] learning_rate=0.05, max_depth=8, min_data=100, min_data_in_leaf=25, n_estimators=100, num_leaves=100 
[CV]  learning_rate=0.05, max_depth=8, min_data=100, min_data_in_leaf=25, n_estimators=100, num_leaves=100, score=0.7842323651452282, total=   0.2s
[CV] learning_rate=0.05, max_depth=8, min_data=100, min_data_in_leaf=25, n_estimators=100, num_leaves=100 
[CV]  learning_rate=0.05, max_depth=8, min_data=100, min_data_in_leaf=25, n_estimators=100, num_leaves=100, score=0.7749169435215947, total=   0.2s
[CV] learning_rate=0.05, max_depth=8, min_data=100, min_data_in_leaf=25, n_estimators=100, num_leaves=120 
[CV]  learning_rate=0.05, max_depth=8, min_data=100, min_data_in_leaf=25, n_estimators=100, num_leaves=120, score=0.7560165975103734, total=   0.2s
[CV] learning_rate=0.05, max_depth=8, min_data=100, min_data_in_leaf=25, n_estimators=1

[CV]  learning_rate=0.05, max_depth=8, min_data=500, min_data_in_leaf=25, n_estimators=100, num_leaves=80, score=0.7560165975103734, total=   0.2s
[CV] learning_rate=0.05, max_depth=8, min_data=500, min_data_in_leaf=25, n_estimators=100, num_leaves=80 
[CV]  learning_rate=0.05, max_depth=8, min_data=500, min_data_in_leaf=25, n_estimators=100, num_leaves=80, score=0.7842323651452282, total=   0.2s
[CV] learning_rate=0.05, max_depth=8, min_data=500, min_data_in_leaf=25, n_estimators=100, num_leaves=80 
[CV]  learning_rate=0.05, max_depth=8, min_data=500, min_data_in_leaf=25, n_estimators=100, num_leaves=80, score=0.7749169435215947, total=   0.1s
[CV] learning_rate=0.05, max_depth=8, min_data=500, min_data_in_leaf=25, n_estimators=100, num_leaves=100 
[CV]  learning_rate=0.05, max_depth=8, min_data=500, min_data_in_leaf=25, n_estimators=100, num_leaves=100, score=0.7560165975103734, total=   0.2s
[CV] learning_rate=0.05, max_depth=8, min_data=500, min_data_in_leaf=25, n_estimators=100, n

[CV]  learning_rate=0.05, max_depth=8, min_data=1000, min_data_in_leaf=25, n_estimators=100, num_leaves=50, score=0.7576763485477178, total=   0.2s
[CV] learning_rate=0.05, max_depth=8, min_data=1000, min_data_in_leaf=25, n_estimators=100, num_leaves=50 
[CV]  learning_rate=0.05, max_depth=8, min_data=1000, min_data_in_leaf=25, n_estimators=100, num_leaves=50, score=0.7775933609958506, total=   0.2s
[CV] learning_rate=0.05, max_depth=8, min_data=1000, min_data_in_leaf=25, n_estimators=100, num_leaves=50 
[CV]  learning_rate=0.05, max_depth=8, min_data=1000, min_data_in_leaf=25, n_estimators=100, num_leaves=50, score=0.7790697674418605, total=   0.1s
[CV] learning_rate=0.05, max_depth=8, min_data=1000, min_data_in_leaf=25, n_estimators=100, num_leaves=80 
[CV]  learning_rate=0.05, max_depth=8, min_data=1000, min_data_in_leaf=25, n_estimators=100, num_leaves=80, score=0.7560165975103734, total=   0.2s
[CV] learning_rate=0.05, max_depth=8, min_data=1000, min_data_in_leaf=25, n_estimators=

[Parallel(n_jobs=1)]: Done 1512 out of 1512 | elapsed:  7.7min finished


 0.7769784172661871 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=5,
        min_child_samples=20, min_child_weight=0.001, min_data=100,
        min_data_in_leaf=30, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=50, objective='binary', random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0, task='train')
ROC AUC score 0.7731313801661968
